In [7]:
# Test recommender as parts are developed
import collections
import libsbml
import numpy as np
import os
import pickle
import pandas as pd
import sys
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

In [5]:
import AMAS

In [8]:
# This time, load a BiGG model
one_biomd = 'BIOMD0000000190.xml'
one_biomd_fpath = os.path.join(BIOMODEL_DIR, one_biomd)
reader = libsbml.SBMLReader()
document = reader.readSBML(one_biomd_fpath)
model = document.getModel()

In [12]:

recom = AMAS.Recommender(libsbml_fpath=one_biomd_fpath)

In [13]:
# spec_list = ['M_ac_c', 'M_adp_c', 'M_atp_c', 'M_actp_c']
# spec_list = ['ATP', 'ADP', 'AMP']
res = recom.getSpeciesAnnotation(pred_id='SAM')

In [14]:
res

Recommendation(id='SAM', credibility=1.0, candidates=[('CHEBI:15414', 1.0), ('CHEBI:59789', 1.0)], urls=['https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI%3A15414', 'https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI%3A59789'])

In [7]:
specs = recom.getSpeciesListAnnotation(pred_list=['SAM', 'ORN'], id=True)
one_res = specs[1]
print(one_res)

Recommendation(id='ORN', credibility=1.0, candidates=[('CHEBI:15729', 1.0), ('CHEBI:46911', 1.0)], urls=['https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI%3A15729', 'https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI%3A46911'])


In [8]:
specs[0]

Recommendation(id='SAM', credibility=1.0, candidates=[('CHEBI:15414', 1.0), ('CHEBI:59789', 1.0)], urls=['https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI%3A15414', 'https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI%3A59789'])

In [9]:
recom.reactions.reaction_components

{'ODC': ['ORN', 'P'],
 'SAMdc': ['SAM', 'A'],
 'SSAT_for_S': ['AcCoA', 'aS', 'S', 'CoA'],
 'SSAT_for_D': ['AcCoA', 'aD', 'D', 'CoA'],
 'PAO_for_aD': ['aD', 'P'],
 'PAO_for_aS': ['D', 'aS'],
 'SpdS': ['D', 'P', 'A'],
 'SpmS': ['D', 'S', 'A'],
 'MAT': ['SAM', 'Met'],
 'VCoA': ['AcCoA', 'CoA'],
 'VacCoA': ['AcCoA', 'CoA'],
 'P_efflux': ['P'],
 'aD_efflux': ['aD']}

In [4]:
two_biomd = 'BIOMD0000000190.xml'
two_biomd_fpath = os.path.join(BIOMODEL_DIR, two_biomd)
recom2 = recommender.Recommender(libsbml_fpath=two_biomd_fpath)
# reac_list = ['SAMdc', 'SSAT_for_S']
one_reac = 'SAMdc'
res2 = recom2.getReactionAnnotation(one_reac)

In [5]:
res2

Recommendation(id='SAMdc', credibility=0.38, candidates=[('RHEA:15984', 0.5)], urls=['https://www.rhea-db.org/rhea/15984'])

In [6]:
reac_list = ['ODC', 'SAMdc']
res3 = recom.getReactionListAnnotation(reac_list)

In [7]:
res3

[Recommendation(id='ODC', credibility=1.0, candidates=[('RHEA:28830', 1.0), ('RHEA:22967', 0.5), ('RHEA:59051', 0.5)], urls=['https://www.rhea-db.org/rhea/28830', 'https://www.rhea-db.org/rhea/22967', 'https://www.rhea-db.org/rhea/59051']),
 Recommendation(id='SAMdc', credibility=0.38, candidates=[('RHEA:15984', 0.5)], urls=['https://www.rhea-db.org/rhea/15984'])]

In [6]:
pred_id =  'ODC'
specs2predict = recom.reactions.reaction_components[pred_id]
spec_results = recom.getSpeciesListAnnotation(specs2predict, id=True)
pred_formulas = dict()
for one_recom in spec_results:
  cands = [val[0] for val in one_recom.candidates]
  forms = list(set([sa.ref_shortened_chebi_to_formula[k] \
           for k in cands if k in sa.ref_shortened_chebi_to_formula.keys()]))
  pred_formulas[one_recom.id] = forms

pred_reaction = recom.reactions.predictAnnotation(inp_spec_dict=pred_formulas,
                                                 inp_reac_list=[pred_id])
pred_score = recom.reactions.evaluatePredictedReactionAnnotation([pred_id])
print(pred_score)

{'ODC': 1.0}


In [7]:
pred_reaction

{'SAMdc': [('RHEA:15984', 0.5)]}

In [8]:
recom.reactions.predictAnnotation(inp_spec_dict=pred_formulas,
                                  inp_reac_list=[pred_id])

{'SAMdc': [('RHEA:15984', 0.5)]}

In [18]:

print(spec_formulas)

{'A': ['C14N6O3S'], 'SAM': ['C15N6O5S']}
